In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn as sns
from os import cpu_count as ncpu

In [2]:
data = pd.read_csv(filepath_or_buffer='./data_normalized.csv',sep=',')
data = data.drop(labels=['Unnamed: 0','MeanScore','MeanScoreDisc'],axis=1)

In [3]:
cols = list(data.columns.values)
cols_target = list(cols[1:4])
cols_target_disc = list(cols[-3:])
cols_var = list(data.drop(labels=cols_target+cols_target_disc,axis=1).columns.values)

# <span style="color:rgb(0,200,200)"> Selección de variables con sklearn</span>

In [4]:
training = data[cols_var]
target = data[cols_target_disc]
display([[training.columns.values],[],[target.columns.values]])
n_classes = 3

[[array(['NrSiblings', 'LunchType', 'WklyStudyHours', 'ParentEduc',
         'TestPrep', 'Gender', 'EthnicGroup', 'ParentMaritalStatus',
         'PracticeSport', 'IsFirstChild', 'TransportMeans'], dtype=object)],
 [],
 [array(['MathScoreDisc', 'ReadingScoreDisc', 'WritingScoreDisc'],
        dtype=object)]]

## <span style="color:rgb(200,0,0)"> Select Kbest </span>

In [5]:
from sklearn.feature_selection import chi2,f_classif, SelectKBest

In [6]:
crit = [f_classif]

for targ in cols_target_disc:
    print(f"\n--- {targ} ---")

    x,y = [np.array(training),(np.array(target[targ]))]
    
    result = []        
    for method in crit:
        selector = SelectKBest(score_func=method,k=n_classes)
        atributes = selector.fit(x,y).get_support()
        atributos = [cols_var[i] for i in list(atributes.nonzero()[0])]
        result.append(atributos)
        
    result = pd.DataFrame(dict(zip(crit,result)),columns=crit)
    display(result)



--- MathScoreDisc ---


,<function f_classif at 0x7fb1697e13a0>
0,LunchType
1,ParentEduc
2,EthnicGroup



--- ReadingScoreDisc ---


,<function f_classif at 0x7fb1697e13a0>
0,LunchType
1,TestPrep
2,Gender



--- WritingScoreDisc ---


,<function f_classif at 0x7fb1697e13a0>
0,LunchType
1,TestPrep
2,Gender


## <span style="color:rgb(200,0,0)">Árboles de desición</span>

In [7]:
from sklearn.feature_selection import RFECV,RFE
from sklearn.ensemble import ExtraTreesRegressor

In [8]:
crit = ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
crit = ['friedman_mse','poisson']

for targ in cols_target_disc:
    print(f"\n--- {targ} ---")

    x,y = [np.array(training),np.array(target[targ])]
    result = []        
    for method in crit:
        print(f"--- method {method} ---")
        model = ExtraTreesRegressor(criterion=method,n_jobs=ncpu())
        selector = RFE(estimator=model,n_features_to_select=n_classes)
        atributes = selector.fit(x,y).support_
        atributos = [cols_var[i] for i in list(atributes.nonzero()[0])]
        result.append(atributos)
        
    result = pd.DataFrame(dict(zip(crit,result)),columns=crit)
    display(result)



--- MathScoreDisc ---
--- method friedman_mse ---
--- method poisson ---


,friedman_mse,poisson
0,LunchType,LunchType
1,ParentEduc,ParentEduc
2,EthnicGroup,EthnicGroup



--- ReadingScoreDisc ---
--- method friedman_mse ---
--- method poisson ---


,friedman_mse,poisson
0,LunchType,LunchType
1,ParentEduc,ParentEduc
2,Gender,Gender



--- WritingScoreDisc ---
--- method friedman_mse ---
--- method poisson ---


,friedman_mse,poisson
0,LunchType,LunchType
1,TestPrep,TestPrep
2,Gender,Gender


### Variables más importantes

In [9]:
cols_var_math = ['']

# <span style="color:rgb(200,200,0)">Modelos</span>

## <span style="color:rgb(0,200,200)">Regresión lineal</span>

## <span style="color:rgb(0,200,200)">Regrsión logísitica</span>

## <span style="color:rgb(0,200,200)">K-Vecinos</span>

In [11]:
from sklearn.neighbors import KNeighborsRegressor

## <span style="color:rgb(0,200,200)">SVM</span> ESTO SE TARDA MUCHOOO